In [1]:
import pandas as pd
import os
import urllib

# Download the data

If `wget` is not installed on your machine, you may want to try `curl URL > file.txt` or the `urllib` package in python.

In [2]:
%%bash
mkdir data_ghcn
cd data_ghcn
#wget https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt
#wget https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2019.csv.gz
#gzip -d 2019.csv.gz

--2021-11-24 10:47:08--  https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt
Resolving www1.ncdc.noaa.gov (www1.ncdc.noaa.gov)... 205.167.25.178, 205.167.25.167, 205.167.25.168, ...
Connecting to www1.ncdc.noaa.gov (www1.ncdc.noaa.gov)|205.167.25.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10277258 (9.8M) [text/plain]
Saving to: ‘ghcnd-stations.txt’

     0K .......... .......... .......... .......... ..........  0%  595K 17s
    50K .......... .......... .......... .......... ..........  0% 1.14M 13s
   100K .......... .......... .......... .......... ..........  1% 77.2M 8s
   150K .......... .......... .......... .......... ..........  1% 1.15M 8s
   200K .......... .......... .......... .......... ..........  2%  111M 7s
   250K .......... .......... .......... .......... ..........  2%  115M 6s
   300K .......... .......... .......... .......... ..........  3%  148M 5s
   350K .......... .......... .......... .......... ..........  3

# Data Preprocessing

In [2]:
def get_vals(line):
    ls = line.split(',')
    station = ls[0]
    time = ls[1]
    val = float(ls[3])
    return [station, time, val]

def get_stations(filename='ghcnd-stations.txt'):
    df = pd.read_csv(filename, '/t', header=None)
    df = df[0].str.split(expand=True)[[0, 1, 2, 3]]
    df.columns = ['Station', 'Latitude', 'Longitude', 'Elevation']
    return df

def process_year(year, stations, col='TAVG', basedir='data_ghcn'):
    tavg = []
    with open(os.path.join(basedir, "%s.csv" % year)) as h:
        l = h.readline()
        while l:
            if col in l:
                v = get_vals(l)
                if v[0] in stations.Station:
                    tavg.append(get_vals(l))
            l = h.readline()
    df_tavg = pd.DataFrame(tavg, columns=['Station', 'Date', col])
    df_merged = df_tavg.merge(stations, left_on='Station', right_on='Station', how='left')
    df_merged['Date'] = df_merged['Date'].apply(pd.Timestamp)
    for c in ['Latitude', 'Longitude', col, 'Elevation']:
        df_merged[c] = df_merged[c].astype(float)
    return df_merged[['Station', 'Date', col, 'Latitude', 'Longitude', 'Elevation']]

In [3]:
stations = get_stations()
df1 = process_year('2019', stations, col='TAVG')
stations = stations[stations.Station.isin(df1.Station)]

/tmp/ipykernel_43/1711228864.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  stations = get_stations()
/opt/conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [4]:
df2 = process_year('2019', stations, col='PRCP')

In [7]:
df = df1.merge(df2[['Station', 'Date', 'PRCP']], on=['Station', 'Date'])
df.to_csv('data_ghcn/daily_global_weather_2019.csv')
df

,TAVG,Latitude,Longitude,Elevation,Station,Date,PRCP


In [6]:
df.head(15)

,TAVG,Latitude,Longitude,Elevation,Station,Date,PRCP
